In [23]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.messages import HumanMessage, ToolMessage
from langgraph.checkpoint.memory import InMemorySaver
import os

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="deepseek/deepseek-v3.2",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [24]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

### Инструмент для записи в состояние

In [25]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour,
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]
    })

In [26]:
agent = create_agent(
    model=llm,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [27]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

print(response['messages'][-1].content)

Your favourite colour has been successfully updated to green!


In [28]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='c4f75d4c-aca1-48d7-965e-bbf9cbef9d60'),
              AIMessage(content="I'll update your favourite colour to green in the system.\n\n", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 330, 'total_tokens': 391, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 0.00010898, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 8.58e-05, 'upstream_inference_completions_cost': 2.318e-05}}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3.2', 'system_fingerprint': None, 'id': 'gen-1767464298-y3iL0LtWORxSxyRFfaM9', '

In [29]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='28dcc3f6-3b82-4f30-a20c-8d5b35c74c3b'),
              AIMessage(content="Hello! I'm doing well, thank you for asking. How are you today? Is there anything I can help you with?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 331, 'total_tokens': 358, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 9.632e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 8.606e-05, 'upstream_inference_completions_cost': 1.026e-05}}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3.2', 'system_fingerprint': None, 'id': 'gen-176

### Инструмент для чтения из состояния

In [30]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """ Read the favourite colour of the user from the state"""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=llm,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='84949005-bf48-4f9e-b5f8-56925d850355'),
              AIMessage(content="I see that you've shared your favorite color is green. Let me update that in the system for you.\n\n", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 126, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 4.797e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 3.276e-05, 'upstream_inference_completions_cost': 1.521e-05}}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3.2', 'system_fingerprint': None, 'id': '

In [31]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='84949005-bf48-4f9e-b5f8-56925d850355'),
              AIMessage(content="I see that you've shared your favorite color is green. Let me update that in the system for you.\n\n", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 126, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 4.797e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 3.276e-05, 'upstream_inference_completions_cost': 1.521e-05}}, 'model_provider': 'openai', 'model_name': 'deepseek/deepseek-v3.2', 'system_fingerprint': None, 'id': '

In [32]:
print(response['messages'][-1].content)

Your favorite color is green!
